In [10]:
# import os
# OPEN_AI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [9]:
# OPEN_AI_API_KEY

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain.utilities import SQLDatabase

In [2]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
llm = ChatOpenAI()#openai_api_key = OPEN_AI_API_KEY)

In [4]:
chain = create_sql_query_chain(llm, db)

In [5]:
response = chain.invoke({"question" : "Get me the name of the employee with the highest salary"})

In [6]:
print(response)

SELECT "name"
FROM "employees"
ORDER BY "salary" DESC
LIMIT 1;


In [13]:
from langchain import OpenAI, LLMMathChain

In [31]:
# llm = OpenAI(openai_api_key=OPEN_AI_API_KEY, temperature=0, model_name = 'gpt-3.5-turbo-instruct')
llm = OpenAI(openai_api_key=OPEN_AI_API_KEY, temperature=0, model = 'gpt-3.5-turbo-instruct')

In [32]:
llm_math = LLMMathChain.from_llm(llm, verbose=True, )

In [33]:
ans = llm_math.run('What is the 3rd root of 1498?')



> Entering new LLMMathChain chain...
What is the 3rd root of 1498?```text
1498**(1/3)
```
...numexpr.evaluate("1498**(1/3)")...

Answer: 11.442052543837162
> Finished chain.


In [34]:
print(ans)

Answer: 11.442052543837162


## 5.4 DALL-E using LLMChain

In [48]:
from langchain.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [58]:
llm = OpenAI(openai_api_key = OPEN_AI_API_KEY, model = 'dall-e-2') #model='gpt-3.5-turbo-instruct', openai_api_key = OPEN_AI_API_KEY)

In [59]:
prompt = PromptTemplate(input_variables = ['image_desc'],
                       template = 'Generate an image baseed on the following description: {image_desc}')

In [60]:
type(prompt)

langchain_core.prompts.prompt.PromptTemplate

In [61]:
chain = LLMChain(llm=llm, prompt = prompt)

In [62]:
prompt = ['folks playing tennis', 'a crying child']

In [63]:
for x in prompt:
    print(DallEAPIWrapper(openai_api_key = OPEN_AI_API_KEY).run(chain.run(x)))

PermissionDeniedError: Error code: 403 - {'error': {'message': 'You are not allowed to sample from this model', 'type': 'invalid_request_error', 'param': None, 'code': None}}

## 5.5 Custom Chains using LCEL

In [64]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [66]:
prompt = ChatPromptTemplate.from_template('what is the city {person} is from?\
Translage: {sentence} in the city native language')

In [68]:
model = ChatOpenAI(openai_api_key = OPEN_AI_API_KEY)

In [69]:
chain = prompt|model|StrOutputParser()

In [70]:
chain.invoke({'person':'Virat Kohli', 'sentence':'how are you?'})

'Virat Kohli is from Delhi. \n\nIn Hindi, "how are you?" can be translated as "आप कैसे हैं?"'

#### Chaining multiple chains

In [72]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

In [74]:
prompt1 = ChatPromptTemplate.from_template('which country won the {game} world cup?')
prompt2 = ChatPromptTemplate.from_template('suggest the best {entity} from {country}')

In [79]:
model = ChatOpenAI(openai_api_key = OPEN_AI_API_KEY, model = 'gpt-3.5-turbo-instruct')

In [86]:
chain1 = prompt1|model|StrOutputParser()

In [87]:
chain1.invoke({'game': 'football'})

NotFoundError: Error code: 404 - {'error': {'message': 'This is not a chat model and thus not supported in the v1/chat/completions endpoint. Did you mean to use v1/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

In [81]:
chain2 = (
    {'country': chain1, 'entity': itemgetter('entity')}
    |prompt2
    |model
    |StrOutputParser()
)

In [83]:
chain2.invoke({'game': 'football', 'entity': 'dish'})

NotFoundError: Error code: 404 - {'error': {'message': 'This is not a chat model and thus not supported in the v1/chat/completions endpoint. Did you mean to use v1/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}